In [64]:
import urllib
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

server='DESKTOP-HJVSCEN\MSSQLSERVER1'
database='Python ETL'
username='sa'
password='Ka@12345678'


ConnectionString = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
# URL-encode the connection string for SQLAlchemy
params=urllib.parse.quote_plus(ConnectionString)

engine=create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [65]:
df_existing=pd.read_sql_table('customers_base',con=engine)
df_existing=df_existing.sort_values(by='customerid').reset_index(drop=True)

In [66]:
df_existing['start_date']=datetime(2025, 6, 20)
df_existing['end_date']=pd.NaT
df_existing['is_active']=True
df_existing

,customerid,name,city,email,lastupdated,start_date,end_date,is_active
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,NaT,True
1,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22,2025-06-20,NaT,True
2,103,John,Pune,john@gmail.com,2025-03-24,2025-06-20,NaT,True
3,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,NaT,True
4,105,Chiru,Banglore,chiru@gmail.com,2025-05-28,2025-06-20,NaT,True
5,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24,2025-06-20,NaT,True
6,107,Ravi,Delhi,ravi@gmail.com,2025-06-20,2025-06-20,NaT,True
7,108,Jack,Delhi,jack@gmail.com,2025-06-20,2025-06-20,NaT,True


In [67]:
df_existing.to_sql('scdtype2',con=engine,index=False,if_exists='replace')

8

In [68]:
scd2_df=pd.read_sql_table('scdtype2',con=engine)
scd2_df

,customerid,name,city,email,lastupdated,start_date,end_date,is_active
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,NaT,True
1,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22,2025-06-20,NaT,True
2,103,John,Pune,john@gmail.com,2025-03-24,2025-06-20,NaT,True
3,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,NaT,True
4,105,Chiru,Banglore,chiru@gmail.com,2025-05-28,2025-06-20,NaT,True
5,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24,2025-06-20,NaT,True
6,107,Ravi,Delhi,ravi@gmail.com,2025-06-20,2025-06-20,NaT,True
7,108,Jack,Delhi,jack@gmail.com,2025-06-20,2025-06-20,NaT,True


Read incoming dataframe

In [69]:
incoming_df = pd.DataFrame([
    {"customerid": 101, "name": "Tanuj", "city": "Bangalore", "email": "tanuj.new@gmail.com", "lastupdated": datetime(2025, 6, 21)},  # Changed city/email
    {"customerid": 103, "name": "John", "city": "Pune", "email": "john@gmail.com", "lastupdated": datetime(2025, 6, 21)},             # No change
    {"customerid": 104, "name": "Smrithi", "city": "Chennai", "email": "smrithi@gmail.com", "lastupdated": datetime(2025, 6, 21)},   # Changed city
    {"customerid": 109, "name": "Riya", "city": "Nagpur", "email": "riya@gmail.com", "lastupdated": datetime(2025, 6, 21)}            # New
])


**Step 3: Load the SCD2 table from database and isolate active records**

In [70]:
scd2_df.loc[6,'is_active']=False

In [71]:
active_df=scd2_df[scd2_df['is_active']==True]
active_df

,customerid,name,city,email,lastupdated,start_date,end_date,is_active
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,NaT,True
1,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22,2025-06-20,NaT,True
2,103,John,Pune,john@gmail.com,2025-03-24,2025-06-20,NaT,True
3,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,NaT,True
4,105,Chiru,Banglore,chiru@gmail.com,2025-05-28,2025-06-20,NaT,True
5,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24,2025-06-20,NaT,True
7,108,Jack,Delhi,jack@gmail.com,2025-06-20,2025-06-20,NaT,True


Merge the existing and incoming df values using inner join to get the common customers present in the incoming and existing data

In [72]:
merged_df=pd.merge(active_df,incoming_df,how='inner',on='customerid',suffixes=['_old',''])
merged_df

,customerid,name_old,city_old,email_old,lastupdated_old,start_date,end_date,is_active,name,city,email,lastupdated
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,NaT,True,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-21
1,103,John,Pune,john@gmail.com,2025-03-24,2025-06-20,NaT,True,John,Pune,john@gmail.com,2025-06-21
2,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,NaT,True,Smrithi,Chennai,smrithi@gmail.com,2025-06-21


From the merged-df find the actual changed values that is the values which are actually having any changes in its new and old values

In [73]:
changed_df=merged_df[
    (merged_df['name_old']!=merged_df['name']) |
    (merged_df['city_old']!=merged_df['city']) |
    (merged_df['email_old']!=merged_df['email'])
]
changed_df

,customerid,name_old,city_old,email_old,lastupdated_old,start_date,end_date,is_active,name,city,email,lastupdated
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,NaT,True,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-21
2,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,NaT,True,Smrithi,Chennai,smrithi@gmail.com,2025-06-21


Get the new versions of the data which needs to be inserted i.e, the values from incoming df which have to be updated for the existing customers

In [74]:
today=pd.to_datetime('today').normalize()
new_versions_df=changed_df[['customerid','name','city','email','lastupdated']].copy()
new_versions_df['start_date']=today
new_versions_df['end_date']=pd.NaT
new_versions_df['is_active']=True
new_versions_df

,customerid,name,city,email,lastupdated,start_date,end_date,is_active
0,101,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-21,2025-06-24,NaT,True
2,104,Smrithi,Chennai,smrithi@gmail.com,2025-06-21,2025-06-24,NaT,True


Now deactivate the customers old data record and insert the end_date . You can use scd2_df and not in active_df because we will be rewriting the scdtype2 table values so as to concatenate in the end. if you do it in active_df at the end of concatenation it is not useful as we will be concatenating existing data(with inactives), new versions of previous data and also the completely new values

In [75]:
scd2_df.loc[scd2_df['customerid'].isin(changed_df['customerid']),'end_date']=today
scd2_df.loc[scd2_df['customerid'].isin(changed_df['customerid']),'is_active']=False

In [76]:
#we should not be using the below method as we have not initiated any indexes for scd2_df and incoming_df
# existing_ids=incoming_df.index.intersection(scd2_df.index)
# new_ids=incoming_df.index.difference(scd2_df.index)
# print('existing ids:',existing_ids)
# print('new ids:',new_ids)


In [79]:
existing_ids=scd2_df['customerid'].unique()
incoming_ids=incoming_df['customerid'].unique()
new_ids=set(incoming_ids)-set(existing_ids)
print(new_ids)

only_new_df=incoming_df[incoming_df['customerid'].isin(new_ids)].copy()
only_new_df['start_date']=today
only_new_df['end_date']=pd.NaT
only_new_df['is_active']=True
only_new_df

{np.int64(109)}


,customerid,name,city,email,lastupdated,start_date,end_date,is_active
3,109,Riya,Nagpur,riya@gmail.com,2025-06-21,2025-06-24,NaT,True


In [80]:
final_df=pd.concat([scd2_df,new_versions_df,only_new_df],ignore_index=True)

In [81]:
final_df.to_sql('customers_scd2',con=engine,index=False,if_exists='replace')

11

In [82]:
final_df=pd.read_sql_table('customers_scd2',con=engine)
final_df

,customerid,name,city,email,lastupdated,start_date,end_date,is_active
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20,2025-06-20,2025-06-24,False
1,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22,2025-06-20,NaT,True
2,103,John,Pune,john@gmail.com,2025-03-24,2025-06-20,NaT,True
3,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26,2025-06-20,2025-06-24,False
4,105,Chiru,Banglore,chiru@gmail.com,2025-05-28,2025-06-20,NaT,True
5,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24,2025-06-20,NaT,True
6,107,Ravi,Delhi,ravi@gmail.com,2025-06-20,2025-06-20,NaT,False
7,108,Jack,Delhi,jack@gmail.com,2025-06-20,2025-06-20,NaT,True
8,101,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-21,2025-06-24,NaT,True
9,104,Smrithi,Chennai,smrithi@gmail.com,2025-06-21,2025-06-24,NaT,True
